In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [2]:
# !pip install seaborn

In [3]:
import findspark
import pandas as pd
import numpy as np

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

import time as t 
from datetime import date, datetime, timedelta
import joblib
from joblib import Parallel, delayed

import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
# import covalent as ct

import subprocess
import re
import random

from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
# import umap.umap_ as umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.backends.backend_pdf import PdfPages
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from matplotlib.backends.backend_pdf import PdfPages


import gc  # import gorbage collector to resolve the problem of restarting kernel due to large table of population loading in RAM to append
from pyspark.sql.functions import year, col

findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']=1

spark = SparkSession.builder \
       .config("spark.ui.port", "4050") \
       .master("local[2]") \
       .appName("MyApp") \
       .config("spark.driver.maxResultSize", "40g")\
       .config("spark.driver.memory", "140g")\
       .getOrCreate()
sc = spark.sparkContext

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Increase the max fields in the string representation of a plan
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set("spark.sql.debug.maxToStringFields", 2000)

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/07 10:32:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data(problem).csv") \
          .createOrReplaceTempView("VINs_data")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")


In [5]:
df_pca = pd.read_csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/final_features_with_pca.csv")


In [6]:
df_pca

,VIN,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,...,PCA_619,PCA_620,PCA_621,PCA_622,PCA_623,PCA_624,PCA_625,PCA_626,PCA_627,PCA_628
0,1M1AW01Y3FM007792,-6.661494,0.777114,-6.214650,3.802999,-1.157347,-0.019652,-1.956263,-1.514594,-1.221088,...,-0.118860,-0.149406,-0.216605,-0.261613,0.028274,0.561532,-0.394180,-0.092431,-0.064494,0.064263
1,1M1AW01Y5FM007793,-6.661494,0.777114,-6.214650,3.802999,-1.157347,-0.019652,-1.956263,-1.514594,-1.221088,...,-0.118860,-0.149406,-0.216605,-0.261613,0.028274,0.561532,-0.394180,-0.092431,-0.064494,0.064263
2,1M1AW01Y7FM007794,-6.661494,0.777114,-6.214650,3.802999,-1.157347,-0.019652,-1.956263,-1.514594,-1.221088,...,-0.118860,-0.149406,-0.216605,-0.261613,0.028274,0.561532,-0.394180,-0.092431,-0.064494,0.064263
3,1M1AW01Y9FM007795,-6.722626,0.718387,-6.184928,3.778406,-1.196368,-0.046089,-1.907394,-1.546106,-1.245109,...,-0.117673,-0.167923,-0.188455,-0.338463,0.095987,0.499925,-0.299827,-0.126692,-0.031913,0.130040
4,1M1AW01Y0FM007796,-6.682986,0.811179,-6.246205,3.794622,-1.148485,-0.024872,-2.008437,-1.510240,-1.216813,...,-0.142258,-0.132636,-0.189965,-0.262150,0.049897,0.608425,-0.389502,-0.103126,-0.094333,0.062288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322173,1M2TE2GC2NM006730,-15.167413,7.538142,12.187195,-0.532006,-2.945807,0.166734,1.358829,1.048509,-0.296116,...,0.121824,-0.005752,-0.049677,-0.210990,0.177553,0.041627,-0.200111,0.119364,-0.305164,-0.034599
322174,1M2TE2GC4NM006731,-15.167413,7.538142,12.187195,-0.532006,-2.945807,0.166734,1.358829,1.048509,-0.296116,...,0.121824,-0.005752,-0.049677,-0.210990,0.177553,0.041627,-0.200111,0.119364,-0.305164,-0.034599
322175,1M2TE2GC8NM006733,-15.167413,7.538142,12.187195,-0.532006,-2.945807,0.166734,1.358829,1.048509,-0.296116,...,0.121824,-0.005752,-0.049677,-0.210990,0.177553,0.041627,-0.200111,0.119364,-0.305164,-0.034599
322176,1M2TE2GC5NM006737,-14.581698,7.735171,11.738383,-0.078591,-2.709326,2.234065,1.191444,0.106790,-0.319594,...,0.373855,-0.021287,-0.190872,0.109553,0.050740,-0.114874,0.028067,-0.010974,-0.137939,0.043146


In [7]:
# Extract PCA feature columns
pca_columns = [col for col in df_pca.columns if col.startswith("PCA_")]
X = df_pca[pca_columns]


# KMeans Clustering

In [8]:
# KMeans Characteristics:
# - Assumes spherical clusters of equal size.
# - Fast and efficient on large datasets.
# - Requires user to pre-define the number of clusters (k).
# - Sensitive to outliers and initialization.
# - Works best on linearly separable, dense data.

# Main Advantages:
# • Very fast and scalable.
# • Easy to implement and interpret.
# • Efficient for compact and well-separated clusters.


In [ ]:
# Find the best number of clusters
# print("Finding the best K for KMeans...")
file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/kmeans_clustering_analysis.txt", "a")
file.writelines("Finding the best K for KMeans...")
file.close()

scores = []
range_k = range(2, 11)

for k in range_k:
    # print(f"KMeans with {k} clusters \n")
    file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/kmeans_clustering_analysis.txt", "a")
    file.writelines(f"KMeans with {k} clusters \n")
    file.close()
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)
    score = silhouette_score(X, labels)
    scores.append(score)
    # print(f"K={k}, Silhouette Score={score:.4f}")
    file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/kmeans_clustering_analysis.txt", "a")
    file.writelines(f"K={k}, Silhouette Score={score:.4f}")
    file.close()

# Determine the best k
# best_k = K_range[silhouette_scores.index(max(silhouette_scores))]    
best_k = range_k[np.argmax(scores)]
# print(f"\nBest K: {best_k}")
file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/kmeans_clustering_analysis.txt", "a")
file.writelines(f"\nBest K: {best_k}")
file.close()

# Final KMeans
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
labels = kmeans.fit_predict(X)
df_pca["cluster"] = labels

# Apply 2D PCA for visualization
pca_2d = PCA(n_components=2)
X_2d = pca_2d.fit_transform(X)
df_pca["PC1"] = X_2d[:, 0]
df_pca["PC2"] = X_2d[:, 1]

# Save results
file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/kmeans_clustering_analysis.txt", "a")
unique, counts = np.unique(labels, return_counts=True)
for cluster_id, count in zip(unique, counts):
    file.writelines(f"Cluster {cluster_id}: {count} points\n")
    # print(f"Cluster {cluster_id}: {count} points\n")
file.close()

    

Finding the best K for KMeans...
KMeans with 2 clusters 

K=2, Silhouette Score=0.1877
KMeans with 3 clusters 

K=3, Silhouette Score=0.1792
KMeans with 4 clusters 

K=4, Silhouette Score=0.2056
KMeans with 5 clusters 

K=5, Silhouette Score=0.1850
KMeans with 6 clusters 

K=6, Silhouette Score=0.1791
KMeans with 7 clusters 



In [ ]:
# Plot clusters in 2D using first 2 PCA components if available
# print("Creating visualization PDF...")
file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/kmeans_clustering_analysis.txt", "a")
file.writelines("Creating visualization PDF...")
file.close()

pdf = PdfPages('/storage/home/yqf5148/work/volvoPennState/hierarchical/kmeans_clusters.pdf')

plt.figure(figsize=(8,6))
plt.scatter(X.iloc[:,0], X.iloc[:,1], c=labels, cmap='viridis', s=10)
plt.title(f'KMeans Clustering (k={best_k})')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
pdf.savefig()
plt.close()

pdf.close()
# print("Done! ✅")
file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/kmeans_clustering_analysis.txt", "a")
file.writelines("Done! ✅")
file.close()

# Gaussian Mixture Model (GMM) Clustering

In [ ]:
# GMM Characteristics:
# - Assumes clusters follow Gaussian distributions (soft clustering).
# - Provides probabilities of belonging to each cluster.
# - More flexible than KMeans (can model elliptical shapes).
# - Still requires number of clusters to be specified.

# Main Advantages:
# • Captures overlapping clusters better than KMeans.
# • Handles cluster shape variations well.
# • Provides soft assignment (probabilistic membership).


In [ ]:
# --- Version 2: Only GMM ---

output_file_gmm = '/storage/home/yqf5148/work/volvoPennState/hierarchical/gmm_clustering_analysis.txt'

# print("Finding the best number of components for GMM...")
file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/gmm_clustering_analysis.txt", "a")
file.writelines("Finding the best number of components for GMM...")
file.close()

scores_gmm = []

for k in range(2, 11):
    gmm = GaussianMixture(n_components=k, random_state=42)
    labels_gmm = gmm.fit_predict(X)
    score = silhouette_score(X, labels_gmm)
    scores_gmm.append(score)
    # print(f"Components={k}, Silhouette Score={score:.4f}")
    file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/gmm_clustering_analysis.txt", "a")
    file.writelines(f"Components={k}, Silhouette Score={score:.4f}")
    file.close()

best_k_gmm = range_k[np.argmax(scores_gmm)]
# print(f"\nBest Components for GMM: {best_k_gmm}")
file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/gmm_clustering_analysis.txt", "a")
file.writelines(f"\nBest Components for GMM: {best_k_gmm}")
file.close()

# Final GMM
gmm = GaussianMixture(n_components=best_k_gmm, random_state=42)
labels_gmm = gmm.fit_predict(X)


# Save results
file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/gmm_clustering_analysis.txt", "a")
unique, counts = np.unique(labels_gmm, return_counts=True)
for cluster_id, count in zip(unique, counts):
    file.writelines(f"Cluster {cluster_id}: {count} points\n")
    # print(f"Cluster {cluster_id}: {count} points\n")
file.close()

In [ ]:
# Plot GMM Clusters
# print("Creating visualization PDF for GMM...")
file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/gmm_clustering_analysis.txt", "a")
file.writelines("Creating visualization PDF for GMM...")
file.close()

pdf = PdfPages('/storage/home/yqf5148/work/volvoPennState/hierarchical/gmm_clusters.pdf')

plt.figure(figsize=(8,6))
plt.scatter(X.iloc[:,0], X.iloc[:,1], c=labels_gmm, cmap='plasma', s=10)
plt.title(f'GMM Clustering (components={best_k_gmm})')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
pdf.savefig()
plt.close()

pdf.close()
# print("All Done! ✅")
file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/gmm_clustering_analysis.txt", "a")
file.writelines("All Done! ✅")
file.close()

# Density Based Spatial Clustering of Applications with Noise (DBSCAN) Clustering

In [ ]:
# DBSCAN Characteristics:
# - Density-based clustering algorithm.
# - Does not require specifying number of clusters.
# - Can find arbitrarily shaped clusters.
# - Identifies noise and outliers (marked as -1).
# - Sensitive to `eps` and `min_samples` parameters.

# Main Advantages:
# • No need to pre-define number of clusters.
# • Robust to noise and outliers.
# • Discovers clusters of varying shape and size.


In [ ]:
# # --- DBSCAN Clustering with Visualization and Output ---


# # Normalize the data
# # print("🔄 Normalizing data using StandardScaler...")
# file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/dbscan-cluster-analysis.txt", "a")
# file.writelines("🔄 Normalizing data using StandardScaler...")
# file.close()

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# # print("✅ Normalization complete.")
# file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/dbscan-cluster-analysis.txt", "a")
# file.writelines("✅ Normalization complete.")
# file.close()

# # Try different eps values to find a good one
# eps_values = [0.3, 0.5, 0.7, 0.9, 1.0, 1.2]
# min_samples = 5
# # print(f"🚀 Starting DBSCAN clustering for eps values: {eps_values}")
# file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/dbscan-cluster-analysis.txt", "a")
# file.writelines(f"🚀 Starting DBSCAN clustering for eps values: {eps_values}")
# file.close()

# output_file = '/storage/home/yqf5148/work/volvoPennState/hierarchical/dbscan-cluster-analysis.txt'
# pdf = PdfPages('/storage/home/yqf5148/work/volvoPennState/hierarchical/dbscan_clusters.pdf')

# with open(output_file, 'a') as f:
#     for eps in eps_values:
#         # print(f"\n🔍 Running DBSCAN with eps={eps}, min_samples={min_samples}...")
#         file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/dbscan-cluster-analysis.txt", "a")
#         file.writelines(f"\n🔍 Running DBSCAN with eps={eps}, min_samples={min_samples}...")
#         file.close()
#         db = DBSCAN(eps=eps, min_samples=min_samples)
#         labels = db.fit_predict(X_scaled)

#         n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
#         n_noise = list(labels).count(-1)

#         if n_clusters > 1:
#             silhouette = silhouette_score(X_scaled, labels)
#         else:
#             silhouette = -1

#         # print(f"📊 EPS={eps} → Clusters: {n_clusters}, Noise: {n_noise}, Silhouette: {silhouette:.4f}")
#         file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/dbscan-cluster-analysis.txt", "a")
#         file.writelines(f"📊 EPS={eps} → Clusters: {n_clusters}, Noise: {n_noise}, Silhouette: {silhouette:.4f}")
#         file.close()

#         f.write(f"\n=== EPS: {eps} ===\n")
#         f.write(f"Estimated number of clusters: {n_clusters}\n")
#         f.write(f"Estimated number of noise points: {n_noise}\n")
#         f.write(f"Silhouette Score: {silhouette:.4f}\n")

#         # Count points in each cluster
#         unique, counts = np.unique(labels, return_counts=True)
#         for cluster_id, count in zip(unique, counts):
#             f.write(f"Cluster {cluster_id}: {count} points\n")

#         # Visualization
#         plt.figure(figsize=(8, 6))
#         unique_labels = set(labels)
#         colors = [plt.cm.tab10(each) for each in np.linspace(0, 1, len(unique_labels))]

#         for k, col in zip(unique_labels, colors):
#             if k == -1:
#                 col = [0, 0, 0, 1]  # Black for noise
#             class_member_mask = (labels == k)
#             xy = X_scaled[class_member_mask]
#             plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#                      markeredgecolor='k', markersize=4, label=f'Cluster {k}')

#         plt.title(f'DBSCAN Clustering (eps={eps}, clusters={n_clusters}, noise={n_noise})')
#         plt.xlabel("Feature 1")
#         plt.ylabel("Feature 2")
#         plt.legend(loc='best')
#         pdf.savefig()
#         plt.close()
#         print(f"📈 Cluster plot for eps={eps} saved to PDF.")

# pdf.close()
# # print("✨ DBSCAN clustering complete. Results written to dbscan_results.txt and dbscan_clusters.pdf")
# file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/dbscan-cluster-analysis.txt", "a")
# file.writelines("✨ DBSCAN clustering complete. Results written to dbscan_results.txt and dbscan_clusters.pdf")
# file.close()

In [ ]:
# # --- DBSCAN eps Finder with Elbow Plot ---

# # Normalize the data
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Compute the nearest neighbors
# min_samples = 5
# neighbors = NearestNeighbors(n_neighbors=min_samples)
# neighbors_fit = neighbors.fit(X_scaled)
# distances, indices = neighbors_fit.kneighbors(X_scaled)

# # Sort the distances (we use the distance to the k-th neighbor)
# distances_k = np.sort(distances[:, -1])

# # Plot the k-distance graph
# plt.figure(figsize=(10, 6))
# plt.plot(distances_k)
# plt.title(f"k-distance Graph (k={min_samples}) for DBSCAN")
# plt.xlabel("Data Points sorted by distance to {}th nearest neighbor".format(min_samples))
# plt.ylabel(f"Distance to {min_samples}th Nearest Neighbor")
# plt.grid(True)
# plt.savefig("/storage/home/yqf5148/work/volvoPennState/hierarchical/dbscan_elbow_plot.pdf")
# plt.show()

# # print("📉 Elbow plot saved as 'dbscan_elbow_plot.pdf'. Choose the point where the curve bends as optimal eps!")
# file = open(f"/storage/home/yqf5148/work/volvoPennState/hierarchical/dbscan-cluster-analysis.txt", "a")
# file.writelines("📉 Elbow plot saved as 'dbscan_elbow_plot.pdf'. Choose the point where the curve bends as optimal eps!")
# file.close()